## Profitable App Profiles for the App Store and Google Play Markets

First Guided Project in dataquest.io - Data Analys in Python.<br>
Exploring two csv datasets:<br>
* **AppleStore.csv** - containing data about approximately seven thousand iOS apps from the App Store.<br> You can download the data set directly from this [link](https://dq-content.s3.amazonaws.com/350/AppleStore.csv). [Data set description](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps)
* **googleplaystore.csv** - containing data about approximately ten thousand Android apps from Google Play.<br> You can download the data set directly from this [link](https://dq-content.s3.amazonaws.com/350/googleplaystore.csv). [Data set description](https://www.kaggle.com/lava18/google-play-store-apps)


## Opening the csv files

In [1]:
import csv
import os

def opening_csv_files(file_path):
    with open(file_path, encoding='utf-8', mode='r') as csv_file:
        data_set = list(csv.reader(csv_file))
        
        data_header = data_set[0]
        data_body = data_set[1:]
        
    return data_header, data_body

# AppleStore data set
ios_header, ios_dataset = opening_csv_files('src\AppleStore.csv')

# Google Play data set
android_header, android_dataset = opening_csv_files('src\googleplaystore.csv')

# Check headers
print('Apple Store header:\n{}\n'.format(ios_header))
print('Google Play header:\n{}\n'.format(android_header))

# Length of data sets
print('# of rows in Apple Store data set: {}'.format(len(ios_dataset)))
print('# of rows in Google Play data set: {}'.format(len(android_dataset)))

Apple Store header:
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']

Google Play header:
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']

# of rows in Apple Store data set: 7197
# of rows in Google Play data set: 10841


## Exploring data sets
* Print some rows
* Print # of rows
* Print # of columns

In [2]:
def explore_data(dataset, slice_start, slice_end, rows_columns_count=False):
    """
    dataset: list
    slice_start: starting row of a data slice
    slice_end: ending row of a data slice
    rows_columns_count: if True prints the number of rows and columns
    """
    if slice_start > slice_end:
        slice_end = slice_start + 1
    
    slice_of_data = dataset[slice_start:slice_end]
    
    for row in slice_of_data:
        print(row)
        print('='*100)
        
    if rows_columns_count:
        print('# of rows: {}'.format(len(dataset)))
        print('# of columns: {}'.format(len(dataset[0])))
        
print('Quick view of Apple Store data set: \n' + '-'*50 )
print(ios_header)
print('\n')
explore_data(ios_dataset, 0, 3, True)

print('\n')
print('Quick view of Google Play data set: \n' + '-'*50 )
print(android_header)
print('\n')
explore_data(android_dataset, 0, 3, True)

Quick view of Apple Store data set: 
--------------------------------------------------
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']
['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']
['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']
# of rows: 7197
# of columns: 16


Quick view of Google Play data set: 
--------------------------------------------------
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver',

## Deleting wrong data
The Google Play data set has a [dedicated discussion section](https://www.kaggle.com/lava18/google-play-store-apps/discussion), and we can see that one of the [discussions outlines an error for row 10472](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015). In order to prevent this kind of errors all the entries field number checked and compared to the # of header columns.

In [3]:
def len_check_and_log(dataset, dataset_id, header_row):
    for row in dataset:
        if len(row) != len(header_row):
            # add entry to log file
            error_log(error_data=row, dataset_id=dataset_id, header_row=header_row)
            print('Data length error in line in dataset - {}:\n{}'.format(dataset_id, row))
            del row
            
def error_log(error_data, dataset_id, header_row):
    if dataset_id == 'android':
        log_entry_file = 'src/android_error_log.csv' 
    elif dataset_id == 'ios':
        log_entry_file = 'src/ios_error_log.csv'
    
    # Check if log file exists, if not create the file and add header
    if os.path.exists(log_entry_file) == False:
        with open(log_entry_file, 'w', newline='') as log_file:
            writer = csv.writer(log_file)
            writer.writerow(header_row)
    
    # Add wrong line to log file
    with open(log_entry_file, 'a', newline='') as log_file:
        log_entries = csv.writer(log_file)
        log_entries.writerow(error_data)
        
len_check_and_log(android_dataset, 'android', android_header)
len_check_and_log(ios_dataset, 'ios', ios_header)

Data length error in line in dataset - android:
['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


## Checking for duplicated entries
In the Google data set checking for duplicated entries by App name, in the Apple data set checking ids.

In [16]:
def check_duplicate_entries(dataset, column_index):
    """
    collect duplicate entries in a list's column
    """
    duplicate_apps = list()
    unique_apps = list()
    
    for row in dataset:
        app_name = row[column_index]
        if app_name in unique_apps:
            duplicate_apps.append(app_name)
        else:
            unique_apps.append(app_name)
    
    return duplicate_apps, unique_apps

def print_duplicate_apps_summary(dataset, dataset_id):
    """
    print a couple of lines from a list, printed text depends on the dataset_id
    """
    if len(dataset) == 0:
        print('No duplicated data in the {} data set.'.format(dataset_id))
        print('-'*100)
        print('\n')
    else:
        print('# of duplicated apps in {} dataset: {}'.format(dataset_id, len(dataset)))
        print('\n')
        print('Example of duplicated entries:\n {}'.format(dataset[:100]))
        print('-'*100)

ios_list_of_duplicated_apps, ios_list_of_unique_apps = check_duplicate_entries(ios_dataset, 0)
android_list_of_duplicated_apps, android_list_of_unique_apps = check_duplicate_entries(android_dataset, 0)

print_duplicate_apps_summary(ios_list_of_duplicated_apps, 'Apple')
print_duplicate_apps_summary(android_list_of_duplicated_apps, 'Google')

No duplicated data in the Apple data set.
----------------------------------------------------------------------------------------------------


# of duplicated apps in Google dataset: 1181


Example of duplicated entries:
 ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software', 'MailChimp - Email, Marketing Automation', 'Crew - Free Messaging and Scheduling', 'Asana: organize team projects', 'Google Analytics', 'AdWords Express', 'Accounting App - Zoho Books', 'Invoice & Time Tracking - Zoho', 'join.me - Simple Meetings', 'Invoice 2go — Professional Invoices and Estimates', 'SignEasy | Sign and Fill PDF and other Documents', 'Quick PDF Scanner + OCR FREE', 'Genius Scan - PDF Scanner', 'Tiny Scanner - PDF Scanner App', '

Before the analysis duplicated entries needs to be removed and only keep one entry per app.<br>
First let's have a look at some of the duplicated entries by having a closer look .

In [14]:
duplicated_entry_examples = list()
for app in android_dataset:
    app_name = app[0]
    if app_name in ['ZOOM Cloud Meetings', 'Instagram', 'Slack', 'Insightly CRM']:
        duplicated_entry_examples.append(app)
        
# sort the example list by App name
duplicated_entry_examples = sorted(duplicated_entry_examples,key=lambda x: x[0])
print(android_header)
for row in duplicated_entry_examples:
    print(row)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Insightly CRM', 'BUSINESS', '3.8', '1383', '51M', '100,000+', 'Free', '0', 'Everyone', 'Business', 'July 12, 2018', '3.24.1', '5.0 and up']
['Insightly CRM', 'BUSINESS', '3.8', '1383', '51M', '100,000+', 'Free', '0', 'Everyone', 'Business', 'July 12, 2018', '3.24.1', '5.0 and up']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '6650991